# Preprocess functions

In [ ]:
import pandas as pd

df=pd.read_csv("/Users/izx_gamex/Desktop/Tuwaiq/capston/araBERT/df.csv")


def clean_text(text: str):
    if not isinstance(text, str):
        # raise TypeError("text must be a string")
        return ""

    # remove urls
    text = re.sub(r"http\S+", " رابط ", text)

    # replace any digit with رقم
    text = re.sub(r"\d+", " رقم ", text)

    # set space before and after any punctuation
    text = re.sub(r"([^\w\s])", r" \1 ", text)

    # remove extra spaces
    text = re.sub(r"\s+", " ", text)

    words = word_tokenize(text)
    text = " ".join([ w for w in words if len(w) > 1 ] )

    return text.lower().strip()



# Add an ID column to the dataframe
# df.insert(0, "Comment_ID", range(1, len(df) + 1))

# Define delimiters for splitting comments
delimiters = ["\n", "،", ".",'"','(',')']

# Function to split text using multiple delimiters
import re
def split_comment(comment):
    if pd.isna(comment):  # Handle NaN values
        return []
    return [segment.strip() for segment in re.split("|".join(map(re.escape, delimiters)), comment) if segment.strip()]

# Apply the function to split comments and explode into separate rows
df_exploded = df.assign(Comments=df["Comments"].apply(split_comment)).explode("Comments")

df_exploded
# Function to further split comments by the first character if it is "و"
def split_by_waw(comment):
    words = comment.split()  # Split by space
    result = []
    temp = []
    
    for word in words:
        if word.startswith("و") and temp:
            result.append(" ".join(temp))  # Store previous chunk
            temp = [word]  # Start new chunk
        else:
            temp.append(word)
    
    if temp:  # Add the last chunk
        result.append(" ".join(temp))
    
    return result

# Apply the function and explode again
df_exploded["Comments"] = df_exploded["Comments"].apply(split_by_waw)
df = df_exploded.explode("Comments").reset_index(drop=True)

df

انصحكم تسجلون اطفالكم


ValueError: cannot insert Comment_ID, already exists

In [ ]:


df['Comments'] = df['Comments'].astype(str)
df_filtered = df[~df['Comments'].str.contains(r'[A-Za-z]', regex=True)]
df = df_filtered

df['Comments'] = df['Comments'].apply(clean_text)
df

/var/folders/8x/lx_l3ybs0yx0nx_wtwwkvy6c0000gn/T/ipykernel_4620/676240213.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Comments'] = df['Comments'].apply(clean_text)


,Comment_ID,Time_Elapsed,Comments,Likes,url
0,1,3m,مركز ممتاز,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
1,1,3m,ومميز بتنوع انشطتهم,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
2,1,3m,الإداريات,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
3,1,3m,والمعلمات كلهم بشوشات,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
4,1,3m,ويهبل تعاملهم مع الاطفال,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
...,...,...,...,...,...
74412,11203,1y,ممتاز ماشاء الله تبارك الله,1.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74413,11204,6m,فيه اقساط,0.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74414,11205,2y,رايع جميل,3.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74415,11206,2y,ما يردون على الهاتف,0.0,https://www.google.com/maps/place/%D9%85%D8%AF...


In [ ]:
import re

# Alternative function to detect if a string contains only emojis
def is_only_emoji(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & pictographs
        "\U0001F680-\U0001F6FF"  # Transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    
    text = text.strip()  # Remove leading/trailing spaces
    return all(emoji_pattern.match(char) for char in text) if text else False

# Function to check for whitespace-only comments
def is_only_whitespace(text):
    return text.strip() == ""  # Check if text is empty after stripping spaces

# Ensure column is string type
df["Comments"] = df["Comments"].astype(str)

# Remove whitespace-only and emoji-only comments
df_filtered = df[~df["Comments"].apply(is_only_whitespace)]  # Remove whitespace-only comments
df_filtered = df_filtered[~df_filtered["Comments"].apply(is_only_emoji)]  # Remove emoji-only comments

df =df_filtered
df

/var/folders/8x/lx_l3ybs0yx0nx_wtwwkvy6c0000gn/T/ipykernel_4620/3440085127.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Comments"] = df["Comments"].astype(str)


,Comment_ID,Time_Elapsed,Comments,Likes,url
0,1,3m,مركز ممتاز,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
1,1,3m,ومميز بتنوع انشطتهم,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
2,1,3m,الإداريات,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
3,1,3m,والمعلمات كلهم بشوشات,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
4,1,3m,ويهبل تعاملهم مع الاطفال,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
...,...,...,...,...,...
74410,11202,2y,اشكر موظفات مركز صغاري لتعاونهم معي,10.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74412,11203,1y,ممتاز ماشاء الله تبارك الله,1.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74413,11204,6m,فيه اقساط,0.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74414,11205,2y,رايع جميل,3.0,https://www.google.com/maps/place/%D9%85%D8%AF...


In [ ]:
import re
import string
import pandas as pd
import arabicstopwords.arabicstopwords as stp

# -----------------------------
# 1. Define Arabic Preprocessing Functions
# -----------------------------

def preprocess_arabic(text):
    """
    Preprocess Arabic text:
      - Remove diacritics (tashkeel)
      - Normalize characters (e.g., different forms of Alef to a standard form)
      - Remove elongation (limit repeated characters)
      - Remove English letters and digits
      - Remove punctuation and extra spaces
    """
    # Ensure text is a string
    text = str(text)
    
    # Remove diacritics (Arabic Tashkeel)
    # Unicode ranges: \u0617-\u061A and \u064B-\u0652
    tashkeel_pattern = r'[\u0617-\u061A\u064B-\u0652]'
    text = re.sub(tashkeel_pattern, '', text)
    
    # Normalize characters: map different forms of Alef to a standard 'ا'
    text = re.sub(r'[إأٱآا]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ؤ', 'ء', text)
    text = re.sub(r'ئ', 'ء', text)
    # Optionally, replace Taa Marbuta with Ha (or leave as is depending on your needs)
    text = re.sub(r'ة', 'ه', text)
    
    # Remove elongation: keep at most two consecutive occurrences of a character
    text = re.sub(r'(.)\1+', r'\1\1', text)
    
    # Remove English letters and digits
    text = re.sub(r'[A-Za-z0-9]+', '', text)
    
    # Remove punctuation (both English and common Arabic punctuation)
    # The string.punctuation covers English punctuation; add Arabic punctuation manually.
    punct_pattern = f"[{re.escape(string.punctuation)}،؟«»؛…“”‘’–—•]"
    text = re.sub(punct_pattern, ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def tokenize_and_remove_stopwords(text):
    """
    Simple tokenization by splitting on whitespace and then remove Arabic stopwords.
    """
    # Tokenize by splitting on spaces
    tokens = text.split()
    
    stop_words = stp.stopwords_list()


# Define your additional stopwords
    extra_stopwords = [
       'مركز', 'اطفال', 'اطفالنا', 'الاطفال', 'رياض', 'انشطه', 'مدرسه',
       'الله', 'علي', 'ممتاز', 'رائع', 'جدا', 'اكثر', 'ماشاءالله',
        'شكرا', 'نوره', 'بنتي', 'ولدي', 'ابني', 'بنت', 'ولد', 'انا', 'عندي', 'عندهم','والله',
        'المدرسه', 'المدرسة','اللي','تبارك','يسعدهم','تقول','اي','للتواصل','ممكن'
      ]
    
    extended_stop_words = stop_words + extra_stopwords

    # Load Arabic stopwords as a set for fast lookup
    extended_stop_words = set(stp.stopwords_list())
    
    # Remove stopwords
    tokens_filtered = [token for token in tokens if token not in extended_stop_words]
    
    # Rejoin tokens into a processed string
    return ' '.join(tokens_filtered)

# -----------------------------
# 2. Load and Preprocess the Dataset
# -----------------------------

# Load your CSV file (update the path if needed)

# Ensure the Comments column is of type string

# Apply the preprocessing functions to the Comments column
df['Comments'] = df['Comments'].apply(preprocess_arabic)
df['Comments'] = df['Comments'].apply(tokenize_and_remove_stopwords)

# Optionally, remove rows with very short comments (length <= 5 characters)
df['length'] = df['Comments'].str.len()
df = df[df['length'] > 5].copy()
df.drop(columns='length', inplace=True)

print("Sample preprocessed comments:")
df

Sample preprocessed comments:


,Comment_ID,Time_Elapsed,Comments,Likes,url
0,1,3m,مركز ممتاز,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
1,1,3m,ومميز بتنوع انشطتهم,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
2,1,3m,الاداريات,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
3,1,3m,والمعلمات بشوشات,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
4,1,3m,ويهبل تعاملهم الاطفال,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...
...,...,...,...,...,...
74409,11201,2y,بارك الله,7.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74410,11202,2y,اشكر موظفات مركز صغاري لتعاونهم,10.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74412,11203,1y,ممتاز ماشاء الله تبارك الله,1.0,https://www.google.com/maps/place/%D9%85%D8%AF...
74414,11205,2y,رايع جميل,3.0,https://www.google.com/maps/place/%D9%85%D8%AF...


In [ ]:
df = df.sample(frac=1, random_state=101)

----------------------------------------------------------------


# sentiment analysis

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Load pre-trained AraBERT model for sentiment analysis
model_name = "aubmindlab/bert-base-arabertv02-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def predict_sentiment(text, neutral_threshold=0.2):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()[0]
    
    # Check if probabilities are close (indicating uncertainty/neutral)
    if abs(probs[0] - probs[1]) < neutral_threshold:
        return "neutral"
    else:
        return "positive" if np.argmax(probs) == 1 else "negative"


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import pandas as pd

# Load data
df=pd.read_csv("/Users/izx_gamex/Desktop/Tuwaiq/capston/araBERT/df.csv")

# Predict sentiment for each comment
df["sentiment"] = df["Comments"].apply(predict_sentiment)


In [23]:
df

,Unnamed: 0,Comment_ID,Time_Elapsed,Comments,Likes,url,Comments_len,topic,prob,sentiment
0,0,1,3m,مركز ممتاز,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,10,-1,0.000000,neutral
1,1,1,3m,ومميز بتنوع انشطتهم,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,19,5,0.944150,neutral
2,2,1,3m,الاداريات,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,9,2,0.800319,negative
3,3,1,3m,والمعلمات بشوشات,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,16,-1,0.000000,neutral
4,4,1,3m,ويهبل تعاملهم الاطفال,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,21,34,1.000000,neutral
...,...,...,...,...,...,...,...,...,...,...
66788,74409,11201,2y,بارك الله,7.0,https://www.google.com/maps/place/%D9%85%D8%AF...,9,-1,0.000000,neutral
66789,74410,11202,2y,اشكر موظفات مركز صغاري لتعاونهم,10.0,https://www.google.com/maps/place/%D9%85%D8%AF...,31,1,0.760745,neutral
66790,74412,11203,1y,ممتاز ماشاء الله تبارك الله,1.0,https://www.google.com/maps/place/%D9%85%D8%AF...,27,17,0.942831,neutral
66791,74414,11205,2y,رايع جميل,3.0,https://www.google.com/maps/place/%D9%85%D8%AF...,9,-1,0.000000,neutral


In [24]:
df["sentiment"].value_counts()

sentiment
neutral     58725
negative     5864
positive     2204
Name: count, dtype: int64

In [ ]:
# df.to_csv("sentiment_preprocess_N.csv")

In [ ]:
import pandas as pd




# Load data of the full comments without preprocessing
df_N=pd.read_csv("/Users/izx_gamex/Desktop/Tuwaiq/capston/df0_574cleaned.csv")

# Predict sentiment for each comment
df_N["sentiment"] = df_N["Comments"].apply(predict_sentiment)


In [20]:
df_N['sentiment'].value_counts()

sentiment
neutral     9693
positive    1095
negative     419
Name: count, dtype: int64

In [ ]:
#df_N.to_csv("sentiment_original_N.csv")

------------------------

# view data


In [2]:
import pandas as pd


In [3]:
test_df=pd.read_csv("/Users/izx_gamex/Desktop/Tuwaiq/capston/sentiment/sentiment_original_N.csv")
M_df=pd.read_csv("/Users/izx_gamex/Desktop/Tuwaiq/capston/sentiment/sentiment_analysis_full.csv")

In [4]:
test_df.head()

,Unnamed: 0,Time_Elapsed,Comments,Likes,url,sentiment
0,0,3m,مركز ممتاز ومميز بتنوع انشطتهم\nالإداريات والم...,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,neutral
1,1,3m,مركز أكثر من رائع 🤍 أنشطة متنوعة و ممتعة للاطف...,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,neutral
2,2,3w,مركز أكثر من ممتاز ، يتميز فريق العمل بالاحترا...,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,positive
3,3,5m,سجلت بناتي بالمركز بصراحه اهتمام ونظافه والأدا...,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,neutral
4,4,6m,مركز الرواد من أجمل مراكز الأطفال طفلي إبراهيم...,1.0,https://www.google.com/maps/place/%D9%85%D8%B1...,neutral


In [5]:
M_df.head()

,Time_Elapsed,Comments,Likes,url,Sentiment
0,3m,مركز ممتاز ومميز بتنوع انشطتهم\nالإداريات والم...,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,Positive
1,3m,مركز أكثر من رائع 🤍 أنشطة متنوعة و ممتعة للاطف...,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,Positive
2,3w,مركز أكثر من ممتاز ، يتميز فريق العمل بالاحترا...,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,Positive
3,5m,سجلت بناتي بالمركز بصراحه اهتمام ونظافه والأدا...,0.0,https://www.google.com/maps/place/%D9%85%D8%B1...,Positive
4,6m,مركز الرواد من أجمل مراكز الأطفال طفلي إبراهيم...,1.0,https://www.google.com/maps/place/%D9%85%D8%B1...,Positive


In [6]:
M_df["Sentiment"].value_counts()

Sentiment
Neutral     7216
Positive    4651
Negative     791
Name: count, dtype: int64

In [12]:
test_df["sentiment"].value_counts()

sentiment
neutral     9693
positive    1095
negative     419
Name: count, dtype: int64